In [ ]:
import pandas as pd
import numpy as np
import random

from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
path = "/content/drive/MyDrive/Colab/all_stations_data.csv" 


In [ ]:
def flags(data):
  flags = set()
  for key in list(filter(lambda k: 'f' in k, data.select_dtypes(include='object').columns)):
    df = data[key].value_counts().rename_axis('unique_values').reset_index(name='counts')
    flags = flags | set(df['unique_values'])
    flags_list = list(flags)
  return flags_list 

def merge_flags(data, flags_list):
  df = pd.DataFrame({'Flags':flags_list})
  for key in list(filter(lambda k: 'f' in k, data.select_dtypes(include='object').columns)):  
    aux_df = data[key].value_counts().rename_axis('Flags').reset_index(name=key)
    df = df.merge(aux_df,how='left', left_on='Flags', right_on='Flags').fillna(0)
  df[list(filter(lambda k: 'f' in k, data.select_dtypes(include='object').columns))] = df[list(filter(lambda k: 'f' in k, data.select_dtypes(include='object').columns))].astype(int)
  return df

def save_to_Drive(path,df,name): 
  route = path+name+'.csv'
  with open(route, 'w', encoding = 'utf-8-sig') as f:
    df.to_csv(f, index=False)

def df_stats(data_stats): 
  data_stats = data_stats.select_dtypes(include=np.number)
 
  stats = pd.concat([
            data_stats.describe().T,
            data_stats.mode(numeric_only=True).iloc[0].rename("Mode"),
            data_stats.var( numeric_only=True).rename("Var"),
            data_stats.sem(numeric_only=True).rename('SE Mean'),
            data_stats.isnull().sum().rename('N*')
            ],
            axis = 1)
  stats = stats.rename(columns={"count": "N",
                            "std": "StdDev",
                            "25%":"Q1",
                            "75%":"Q3",
                            "50%":"Median",
                            "min":"Minimum",
                            "max":"Maximum",
                            "mean":"Mean"
                            })
  column_names = ["N",
                  "N*",
                  "Mean", 
                  "SE Mean", 
                  "StdDev", 
                  "Var", 
                  "Minimum", 
                  "Maximum", 
                  "Q1", 
                  "Median", 
                  "Q3", 
                  "Mode"]
  return stats.reindex(columns=column_names)

def apply_flags(df):

  for i in df.loc[df['wd_SO2f'] =='P'].index:
    df.wd_SO2f[i] = 0
  for i in df.loc[df['wd_CEf'] =='D'].index:
    df.wd_CEf[i] = 0  
  for key in list(df.select_dtypes(include='number').columns):
    df[key+'f'] = df[key+'f'].fillna(0)
    for i in df.loc[df[key+'f'] != 0].index:
      df[key][i] = np.nan 
    df = df.drop(key+'f', axis=1)

  return df

In [ ]:
data = pd.read_csv(path)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (50,52,54,56,58,60,62,64,66,68,70,72,74,76,78) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
flags_list = flags(data)

In [ ]:
flags = merge_flags(data, flags_list)

In [ ]:
path = "/content/drive/MyDrive/Colab/"
save_to_Drive(path, flags, "flags")

In [ ]:
clean_data = apply_flags(data)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
clean_data.columns

Index(['date', 'pm2.5_SE', 'pm2.5_CE', 'pm2.5_SO2', 'pm10_SE', 'pm10_CE',
       'pm10_SO2', 'so2_SE', 'so2_CE', 'so2_SO2', 'o3_SE', 'o3_CE', 'o3_SO2',
       'co_SE', 'co_CE', 'co_SO2', 'no_SE', 'no_CE', 'no_SO2', 'no2_SE',
       'no2_CE', 'no2_SO2', 'nox_SE', 'nox_CE', 'nox_SO2', 'pm10_NTE2',
       'pm2.5_NTE2', 'o3_NTE2', 'no_NTE2', 'no2_NTE2', 'nox_NTE2', 'so2_NTE2',
       'co_NTE2', 'prs_NTE2', 'rh_NTE2', 'tout_NTE2', 'sr_NTE2', 'rainf_NTE2',
       'ws_NTE2', 'wd_NTE2', 'rainf_SE', 'rainf_CE', 'rainf_SO2', 'sr_SE',
       'sr_CE', 'sr_SO2', 'rh_SE', 'rh_CE', 'rh_SO2', 'tout_SE', 'tout_CE',
       'tout_SO2', 'prs_SE', 'prs_CE', 'prs_SO2', 'ws_SE', 'ws_CE', 'ws_SO2',
       'wd_SE', 'wd_CE', 'wd_SO2'],
      dtype='object')

In [ ]:
path = "/content/drive/MyDrive/Colab/"
save_to_Drive(path, clean_data, "clean_data")

In [ ]:
df_stats(clean_data)

,N,N*,Mean,SE Mean,StdDev,Var,Minimum,Maximum,Q1,Median,Q3,Mode
pm2.5_SE,32802.0,6592,18.443387,0.068020,12.319400,151.767614,2.01,349.000,9.99000,15.390,23.690,8.000
pm2.5_CE,28882.0,10512,23.941130,0.092254,15.678303,245.809180,2.17,225.370,12.61000,20.000,31.240,9.000
pm2.5_SO2,20458.0,18936,17.461181,0.853129,122.024226,14889.911821,-9999.00,156.000,9.00000,16.000,25.000,11.000
pm10_SE,37422.0,1972,50.225316,0.167839,32.468143,1054.180281,2.00,706.550,28.00000,43.710,64.000,28.000
pm10_CE,37712.0,1682,52.204350,0.177788,34.525628,1192.019012,2.00,735.000,29.00000,45.000,67.000,29.000
pm10_SO2,36920.0,2474,59.727979,0.330340,63.473489,4028.883748,-9999.00,714.000,37.00000,52.000,75.000,39.000
so2_SE,31663.0,7731,7.323990,0.020896,3.718334,13.826010,0.00,67.600,4.50000,6.800,9.700,4.100
so2_CE,20485.0,18909,4.294533,0.018123,2.593839,6.728003,0.50,61.900,2.70000,3.500,5.200,3.400
so2_SO2,29449.0,9945,3.865748,0.339930,58.334423,3402.904861,-9999.00,67.400,2.90000,3.600,4.800,3.200
o3_SE,37111.0,2283,27.249252,0.096294,18.550271,344.112542,1.00,139.000,12.00000,24.000,39.000,3.000


In [ ]:
for col in list(clean_data.select_dtypes(include='number').columns):
  for i in clean_data.loc[clean_data[col] < -98].index:
      clean_data[col][i] = np.nan

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
df_stats(clean_data)

,N,N*,Mean,SE Mean,StdDev,Var,Minimum,Maximum,Q1,Median,Q3,Mode
pm2.5_SE,32802.0,6592,18.443387,0.068020,12.319400,151.767614,2.010,349.000,9.9900,15.390,23.690,8.000
pm2.5_CE,28882.0,10512,23.941130,0.092254,15.678303,245.809180,2.170,225.370,12.6100,20.000,31.240,9.000
pm2.5_SO2,20455.0,18939,18.930229,0.092377,13.211835,174.552574,2.000,156.000,9.0000,16.000,25.000,11.000
pm10_SE,37422.0,1972,50.225316,0.167839,32.468143,1054.180281,2.000,706.550,28.0000,43.710,64.000,28.000
pm10_CE,37712.0,1682,52.204350,0.177788,34.525628,1192.019012,2.000,735.000,29.0000,45.000,67.000,29.000
pm10_SO2,36919.0,2475,60.000433,0.186803,35.892953,1288.304079,2.000,714.000,37.0000,52.000,75.000,39.000
so2_SE,31663.0,7731,7.323990,0.020896,3.718334,13.826010,0.000,67.600,4.5000,6.800,9.700,4.100
so2_CE,20485.0,18909,4.294533,0.018123,2.593839,6.728003,0.500,61.900,2.7000,3.500,5.200,3.400
so2_SO2,29448.0,9946,4.205427,0.013065,2.242032,5.026707,-0.700,67.400,2.9000,3.600,4.800,3.200
o3_SE,37111.0,2283,27.249252,0.096294,18.550271,344.112542,1.000,139.000,12.0000,24.000,39.000,3.000


In [ ]:
for col in list(filter(lambda k: 'so2' in k, clean_data.columns)):
  for i in clean_data.loc[clean_data[col] < 0].index:
      clean_data[col][i] = np.nan

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
df_stats(clean_data)

,N,N*,Mean,SE Mean,StdDev,Var,Minimum,Maximum,Q1,Median,Q3,Mode
pm2.5_SE,32802.0,6592,18.443387,0.068020,12.319400,151.767614,2.010,349.000,9.9900,15.390,23.690,8.000
pm2.5_CE,28882.0,10512,23.941130,0.092254,15.678303,245.809180,2.170,225.370,12.6100,20.000,31.240,9.000
pm2.5_SO2,20455.0,18939,18.930229,0.092377,13.211835,174.552574,2.000,156.000,9.0000,16.000,25.000,11.000
pm10_SE,37422.0,1972,50.225316,0.167839,32.468143,1054.180281,2.000,706.550,28.0000,43.710,64.000,28.000
pm10_CE,37712.0,1682,52.204350,0.177788,34.525628,1192.019012,2.000,735.000,29.0000,45.000,67.000,29.000
pm10_SO2,36919.0,2475,60.000433,0.186803,35.892953,1288.304079,2.000,714.000,37.0000,52.000,75.000,39.000
so2_SE,31663.0,7731,7.323990,0.020896,3.718334,13.826010,0.000,67.600,4.5000,6.800,9.700,4.100
so2_CE,20485.0,18909,4.294533,0.018123,2.593839,6.728003,0.500,61.900,2.7000,3.500,5.200,3.400
so2_SO2,29447.0,9947,4.205593,0.013065,2.241888,5.026060,0.500,67.400,2.9000,3.600,4.800,3.200
o3_SE,37111.0,2283,27.249252,0.096294,18.550271,344.112542,1.000,139.000,12.0000,24.000,39.000,3.000


In [ ]:
for col in list(filter(lambda k: 'prs' in k, clean_data.columns)):
  for i in clean_data.loc[clean_data[col] < 400].index:
      clean_data[col][i] = np.nan



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
for col in list(filter(lambda k: 'sr' in k, clean_data.columns)):
  for i in clean_data.loc[clean_data[col] > 10].index:
      clean_data[col][i] = np.nan



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:

final_clean_data = clean_data[list(filter(lambda k: 'NTE2' not in k, clean_data.columns))]


In [ ]:
df_stats(final_clean_data)

,N,N*,Mean,SE Mean,StdDev,Var,Minimum,Maximum,Q1,Median,Q3,Mode
pm2.5_SE,32802.0,6592,18.443387,0.068020,12.319400,151.767614,2.010,349.000,9.9900,15.390,23.69000,8.000
pm2.5_CE,28882.0,10512,23.941130,0.092254,15.678303,245.809180,2.170,225.370,12.6100,20.000,31.24000,9.000
pm2.5_SO2,20455.0,18939,18.930229,0.092377,13.211835,174.552574,2.000,156.000,9.0000,16.000,25.00000,11.000
pm10_SE,37422.0,1972,50.225316,0.167839,32.468143,1054.180281,2.000,706.550,28.0000,43.710,64.00000,28.000
pm10_CE,37712.0,1682,52.204350,0.177788,34.525628,1192.019012,2.000,735.000,29.0000,45.000,67.00000,29.000
pm10_SO2,36919.0,2475,60.000433,0.186803,35.892953,1288.304079,2.000,714.000,37.0000,52.000,75.00000,39.000
so2_SE,31663.0,7731,7.323990,0.020896,3.718334,13.826010,0.000,67.600,4.5000,6.800,9.70000,4.100
so2_CE,20485.0,18909,4.294533,0.018123,2.593839,6.728003,0.500,61.900,2.7000,3.500,5.20000,3.400
so2_SO2,29447.0,9947,4.205593,0.013065,2.241888,5.026060,0.500,67.400,2.9000,3.600,4.80000,3.200
o3_SE,37111.0,2283,27.249252,0.096294,18.550271,344.112542,1.000,139.000,12.0000,24.000,39.00000,3.000


In [ ]:
save_to_Drive(path, final_clean_data, 'final_all_stations')